In  this notebook , we will procede to metarelation extraction base on a lllm pipeline

prompt used :

You are an helpful assistant helping building an ontology from technical documentation.

Extract the most meaningful keywords of the following text. Keep only keywords that could be concepts and not relations. Write them as a python list of string with double quotes.

Here is an example. Text: This python package is about ontology learning. I do not know a lot about this field. ["python package", "ontology learning", "field"]

Text: {corpus}

In [1]:
import spacy
from typing import Set, List
import pandas as pd
from olaf import Pipeline
from olaf.commons.logging_config import logger
from olaf.data_container import CandidateTerm, Relation, Concept
from olaf.data_container.knowledge_representation_schema import KnowledgeRepresentation
from olaf.pipeline.pipeline_component.term_extraction import (
    POSTermExtraction,
    TFIDFTermExtraction,
    ManualCandidateTermExtraction
    )
from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    CTsToRelationExtraction,
    SynonymRelationExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringRelationExtraction,
    AgglomerativeClusteringConceptExtraction
)
from olaf.commons.spacy_processing_tools import is_not_punct, is_not_stopword, select_on_pos

from olaf.pipeline.pipeline_component.candidate_term_enrichment import SemanticBasedEnrichment

from olaf.repository.corpus_loader.text_corpus_loader import TextCorpusLoader

/home/oumar/Bureau/ontology-learning/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib_venn import venn2, venn3

In [3]:
nlp = spacy.load("en_core_web_lg")

In [11]:
import torch, gc
def free_gpu():
    """
    frees up the GPU cache for other calculations. 
    """
    gc.collect()
    torch.cuda.empty_cache()

free_gpu()

# Select and create relevent concepts from the corpus.


In [5]:
expected_concepts = []
with open("../data/concepts_v2.txt", 'r') as f:
    lines = f.readlines()
    expected_concepts = [concept.rstrip("\n") for concept in lines]
    expected_concepts = [Concept(concept) for concept in expected_concepts]
    f.close()

print(expected_concepts)


[Punching, Welding line, Crescent Gap, Water spot, Oil spot, Silk spot, Inclusion, Rolled pit, Crease, Waist folding, metal surface defect, mechanical failure, drying, mechanical lubricant, temperature, pressure, work roll damage, tension roll damage, local yield, low-carbon, steel strip, folds, rough pockmarked surfaces, metal surface defect, deformation]


In [6]:
corpus_path = "../data/GC10-DET_doc.txt"
corpus_loader = TextCorpusLoader(corpus_path)

In [14]:
free_gpu()

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Testing concept ratio function

In [16]:


def hg_lm_similaritiry(embedding_a : str, embedding_b: str, threshold=.8):
    """compute the cosine similarity between two vectors
    """
    return util.pytorch_cos_sim(embedding_a, embedding_b) > threshold

def create_concepts_embedings(concepts: List[Concept], model=model) -> List[np.ndarray]:
    """
    Create a embedding list of concepts label from a list of Concepts.
    """
    concept_labels = [concept.label for concept in concepts]
    concept_embedings = model.encode(concept_labels)
    return concept_embedings


def get_unexpected_concepts(concepts: List[Concept], expected_concepts : List[Concept]):
    """
    Return all the concepts obtained that do not appear in those expected.
    """
    unexpected_concepts = []
    concepts = list(concepts)
    concepts_embedings = create_concepts_embedings(concepts)
    expected_concepts_embeding = create_concepts_embedings(expected_concepts)
    for idc, concept_embeding in enumerate(concepts_embedings):
        if not any([hg_lm_similaritiry(concept_embeding, expected_concept_embeding) > 0.7 for expected_concept_embeding in expected_concepts_embeding]):
            unexpected_concepts.append(concepts[idc])
    return unexpected_concepts

def get_concept_ratio(pipeline : Pipeline, expected_concepts : List[Concept], comparator = hg_lm_similaritiry, comparator_args:dict={}, verbose=False) -> tuple:
    """
    Calculate the ratio of expected and unexpected concepts in a given pipeline.

    Parameters
    ----------
    pipeline : Pipeline
        The pipeline object containing concepts.
    expected_concepts : List[Concept]
        A list of expected concepts.kwargs

    Returns
    -------
    Tuple[float, float]: A tuple containing:
        The percentage of expected concepts found in the pipeline.
        The percentage of unexpected concepts in the pipeline.
    """
    
    
    found_concepts = list(pipeline.kr.concepts)
    if not found_concepts:
        pipeline.run()
        found_concepts = list(pipeline.kr.concepts)

    if not found_concepts:
        return (0, 0, 0)
    found_concepts_embedings = create_concepts_embedings(found_concepts)
    expected_concepts_embeding = create_concepts_embedings(expected_concepts)
    expected_concept_occ = 0
    for idx, expected_concept_embeding in enumerate(expected_concepts_embeding):
        if verbose:
            print()
            print(f"{expected_concepts[idx]} : ", end="")
        for idc, found_concept_embeding in enumerate(found_concepts_embedings):
            if comparator(expected_concept_embeding, found_concept_embeding, **comparator_args):
                expected_concept_occ += 1
                if verbose:
                    print(f"{found_concepts[idc]} ", end="")
                break 

    recall = expected_concept_occ/len(expected_concepts)
    precision = expected_concept_occ/len(found_concepts)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)
    
comparator_args = {"threshold":.7}

In [18]:
from olaf.pipeline.pipeline_component.term_extraction.manual_candidate_terms import (
    ManualCandidateTermExtraction,
)
from olaf.pipeline.pipeline_component.concept_relation_extraction.candidate_terms_to_concepts import CTsToConceptExtraction

from olaf.pipeline.pipeline_component.concept_relation_extraction.candidate_terms_to_relations import CTsToRelationExtraction


# a list of candidate terms by a LLM to create concepts
concepts = [
    "defect type", 
    "steel strip surface", 
    "punching", 
    "mechanical failure", 
    "production line", 
    "product specifications", 
    "punching defects", 
    "welding line", 
    "coil", 
    "weld line", 
    "crescent gap", 
    "cutting", 
    "water spot", 
    "drying", 
    "product", 
    "processes", 
    "oil spot", 
    "contamination", 
    "mechanical lubricant", 
    "appearance", 
    "silk spot", 
    "plaque", 
    "strip surface", 
    "density", 
    "temperature", 
    "roller", 
    "pressure", 
    "inclusion", 
    "metal surface", 
    "spots", 
    "fish scale shape", 
    "strip shape", 
    "block irregular distribution", 
    "rough pockmarked surfaces", 
    "plate", 
    "rolled pit", 
    "bulges", 
    "pits", 
    "steel plate", 
    "work roll", 
    "tension roll", 
    "damage", 
    "crease", 
    "fold", 
    "spacing", 
    "local yield", 
    "uncoiling process", 
    "waist folding", 
    "wrinkles", 
    "local deformation", 
    "low-carbon"
]

# a list of candidate terms by a LLM to create relations
relations = [
    "described",
    "explaining",
    "appears",
    "leads",
    "resulting",
    "changed",
    "produced",
    "drying",
    "caused",
    "affect",
    "appearing",
    "lies",
    "distributed",
    "accompanied",
    "showing",
    "pressed",
    "occurred",
    "circumvented",
    "detected",
    "tracked",
    "results",
    "like",
    "mainly",
    "uncoiling"
]

ct_concept_label = { concept : {concept} for concept in concepts}

manuel_concept_extraction = ManualCandidateTermExtraction(
    ct_label_strings_map=ct_concept_label
)


# Usefull function

In [19]:
def display_concept(kr : KnowledgeRepresentation) -> None:
    """
    Display the concepts contained in a KnowledgeRepresentation

    Parameters
    ----------
    kr : KnowledgeRepresentation
        The Knowledge Representation containing the concepts and relations.

    Returns
    -------
    None.
    """
    print("Concepts in KR:")
    for concept in kr.concepts:
        print(concept.label)


def display_relation(kr: KnowledgeRepresentation) -> None:
    """
    Display the relations contained in a KnowledgeRepresentation

    Parameters
    ----------
    kr : KnowledgeRepresentation
        The Knowledge Representation containing the concepts and relations.

    Returns
    -------
    None.
    """
    print("Relations in KR:")
    for relation in kr.relations:
        if (
            relation.source_concept is not None
            or relation.destination_concept is not None
        ):
            print(
                (
                    relation.source_concept.label,
                    relation.label,
                    relation.destination_concept.label,
                )
            )

def describe_pipeline(pipeline: Pipeline) -> None:
    """
    Display all pipeline components class names

    Parameters
    ----------
    pipeline : Pipeline
        the pipeline to describe.

    Returns
    -------
    None.
    """
    print(pipeline.__class__.__name__)
    for component in pipeline.pipeline_components:
        print(f"\t {component.__class__.__name__}")

def display_metarelation(kr: KnowledgeRepresentation) -> None:
    """
    Display the relations contained in a KnowledgeRepresentation

    Parameters
    ----------
    kr : KnowledgeRepresentation
        The Knowledge Representation containing the concepts and metarelations.

    Returns
    -------
    None.
    """
    print(f"{len(kr.metarelations)} MetaRelations in KR:")
    for metarelation in kr.metarelations:
        print(metarelation.source_concept,  metarelation, metarelation.destination_concept)



# Creating pipelines

In [20]:
from olaf.pipeline.pipeline_component.term_extraction import (
    ManualCandidateTermExtraction,
    POSTermExtraction,
    TFIDFTermExtraction,
    CvalueTermExtraction
)

from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringConceptExtraction
)


In [21]:
concept_extraction_components = ["CandidatToConcept", "SynonymToConcept", "AgglomerativeClustering"]
term_extraction_components = ["LLM Term Extraction", ]
results = pd.DataFrame(
    index=concept_extraction_components,
    columns=term_extraction_components
    )

multi_index = pd.MultiIndex.from_product([
   concept_extraction_components, 
    ["Precision", "Rappel", "F1"]
    ])
pipelines_scores = pd.DataFrame(index=term_extraction_components, columns=multi_index)

pipelines_scores

CandidatToConcept             SynonymToConcept         \
                            Precision Rappel   F1        Precision Rappel   
LLM Term Extraction               NaN    NaN  NaN              NaN    NaN   

                         AgglomerativeClustering              
                      F1               Precision Rappel   F1  
LLM Term Extraction  NaN                     NaN    NaN  NaN

## LLM Term  Extraction

In [22]:
llm_pipelines = []
llm_results = []

### LLM Term  Extraction and Candidat To Concept Extraction

In [23]:


ct_concept_label = {concept : {concept} for concept in concepts}


llm_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            CTsToConceptExtraction(),
        ],
        corpus_loader=corpus_loader
    )
)
current_pipeline = llm_pipelines[-1]
# current_pipeline.run()


llm_results += list( 
    results:=get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)
    )
print(results)

(0.5, 1.0, 0.6666666666666666)


### LLM Term  Extraction and Synonym Concept Extraction

In [ ]:

llm_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            SemanticBasedEnrichment(
                threshold=.9
            ),
            SynonymConceptExtraction(),
        ],
            corpus_loader=corpus_loader
    )
)
current_pipeline = llm_pipelines[-1]


llm_results += list(
    results:=get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)
    )
print(results)

(0.5, 1.0, 0.6666666666666666)


### LLM Term  Extraction and Agglomerative Clustering Concept Extraction

In [ ]:

llm_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            ),
        ],
        corpus_loader=corpus_loader
    )
)
current_pipeline = llm_pipelines[-1]



llm_results += list(
    results:=get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)
    )
print(results)

[2024-07-02 10:38:28,689] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for embedding_model parameter, default will be set to all-mpnet-base-v2.]
[2024-07-02 10:38:28,690] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for metric option, default will be set to cosine.]


(0.5319148936170213, 1.0, 0.6944444444444444)


In [ ]:
free_gpu()

## debugging LLMPipeline

In [ ]:
agg_pipeline = llm_pipelines[-1]
get_concept_ratio(agg_pipeline, expected_concepts, comparator_args=comparator_args, verbose=True)


Punching : punching 
Welding line : weld line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : fold 
metal surface defect : metal surface 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : temperature 
pressure : pressure 
work roll damage : work roll 
tension roll damage : tension roll 
local yield : local yield 
low-carbon : low-carbon 
steel strip : steel strip surface 
folds : fold 
rough pockmarked surfaces : rough pockmarked surfaces 
metal surface defect : metal surface 
deformation : local deformation 

(0.5319148936170213, 1.0, 0.6944444444444444)

In [ ]:
syn_pipeline = llm_pipelines[1]
get_concept_ratio(syn_pipeline, expected_concepts, comparator_args=comparator_args, verbose=True)


Punching : punching 
Welding line : welding line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : fold 
metal surface defect : metal surface 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : temperature 
pressure : pressure 
work roll damage : work roll 
tension roll damage : tension roll 
local yield : local yield 
low-carbon : low-carbon 
steel strip : steel strip surface 
folds : fold 
rough pockmarked surfaces : rough pockmarked surfaces 
metal surface defect : metal surface 
deformation : local deformation 

(0.5, 1.0, 0.6666666666666666)

In [ ]:
ct_pipeline = llm_pipelines[0]
get_concept_ratio(ct_pipeline, expected_concepts, comparator_args=comparator_args, verbose=True)

### Score des pipelines

In [ ]:
pipelines_scores.loc[term_extraction_components[0]] = llm_results
pipelines_scores

CandidatToConcept                  SynonymToConcept  \
                            Precision Rappel        F1        Precision   
LLM Term Extraction               0.5    1.0  0.666667              0.5   

                                     AgglomerativeClustering                   
                    Rappel        F1               Precision Rappel        F1  
LLM Term Extraction    1.0  0.666667                0.531915    1.0  0.694444

In [ ]:
def create_bar_chart(index_name, data):
    precision = data.loc[index_name].xs('Precision', level=1)
    rappel = data.loc[index_name].xs('Rappel', level=1)
    f1 = data.loc[index_name].xs('F1', level=1)

    fig = go.Figure(data=[
        go.Bar(name='Précision', x=data.columns.levels[0], y=precision),
        go.Bar(name='Rappel', x=data.columns.levels[0], y=rappel),
        go.Bar(name='F1', x=data.columns.levels[0], y=f1)
    ])
    
    # Modifier la disposition du graphique
    fig.update_layout(
        title=f'Scores de Précision, Rappel et F1 pour {index_name}',
        xaxis_title='Composants',
        yaxis_title='Scores',
        barmode='group'
    )
    
    fig.show()

data_long = pipelines_scores.reset_index().melt(id_vars='index', var_name=['Composant', 'Métrique'], value_name='Score')
data_long.rename(columns={'index': 'Extraction'}, inplace=True)
data_long = data_long.dropna()

def create_bar_chart(index_name, data):
    df = data[data['Extraction'] == index_name]
    fig = px.bar(df, x='Composant', y='Score', color='Métrique', barmode='group',
                 title=f'Scores de Précision, Rappel et F1 pour {index_name}')
    
    fig.update_layout(
        xaxis_title='Composants',
        yaxis_title='Scores'
    )
    fig.update_layout(width=1000, height=600)
    fig.show()

# Créer un diagramme en barres pour chaque index
for idx in term_extraction_components:
    create_bar_chart(idx, data_long)

# Hierarchisation

In [24]:
from olaf.pipeline.pipeline_component.concept_relation_hierarchy import SubsumptionHierarchisation

## usefull fonctions

In [35]:
known_truth = {
    "defect": [
        "Punching",
		"Welding line",
		"Crescent Gap",
		"Water spot",
		"Oil spot",
		"Silk spot",
		"Inclusion",
		"Rolled pit",
		"Crease",
		"Waist folding",
    ],
    "cause": [
        "mechanical failure",
		"drying",
		"mechanical lubricant",
		"temperature",
		"pressure",
		"work roll damage",
		"tension roll damage",
		"local yield",
		"low-carbon"
    ],
    "spots": [
        "periodic bulges",
        "pits",
		"rough pockmarked surfaces"
	]
}

def create_exp_meta(known_truth):
	result = set()
	for key, values in known_truth.items():
		for value in values:
			result.add(Metarelation(
				Concept(value),
				Concept(key),
				"is_generalised_by"
			))
	return result

expected_metarelations = create_exp_meta(known_truth)
expected_metarelations

{meta : (rough pockmarked surfaces, is_generalised_by, spots),
 meta : (Punching, is_generalised_by, defect),
 meta : (Waist folding, is_generalised_by, defect),
 meta : (work roll damage, is_generalised_by, cause),
 meta : (tension roll damage, is_generalised_by, cause),
 meta : (Rolled pit, is_generalised_by, defect),
 meta : (periodic bulges, is_generalised_by, spots),
 meta : (temperature, is_generalised_by, cause),
 meta : (Crease, is_generalised_by, defect),
 meta : (drying, is_generalised_by, cause),
 meta : (Welding line, is_generalised_by, defect),
 meta : (low-carbon, is_generalised_by, cause),
 meta : (mechanical failure, is_generalised_by, cause),
 meta : (pits, is_generalised_by, spots),
 meta : (Inclusion, is_generalised_by, defect),
 meta : (pressure, is_generalised_by, cause),
 meta : (Oil spot, is_generalised_by, defect),
 meta : (mechanical lubricant, is_generalised_by, cause),
 meta : (Water spot, is_generalised_by, defect),
 meta : (Silk spot, is_generalised_by, def

In [34]:

def hg_lm_similaritiry(embedding_a : np.ndarray, embedding_b: np.ndarray, threshold=.6):
    return util.pytorch_cos_sim(embedding_a, embedding_b) > threshold


def get_metarelation_ratio(metarelations, expected_metarelations):

    """
    Compte le nombre total de cooccurrences basées sur la similarité cosinus des éléments des tuples.

    Args:
    metarelations (list of tuples): La première liste de tuples contenant des embeddings.
    expected_metarelations (list of tuples): La deuxième liste de tuples contenant des embeddings.

    Returns:
    int: Le nombre total de cooccurrences.
    """
    cooccurrence_count = 0
    
    for t1 in metarelations:
        for t2 in expected_metarelations:
            if all(hg_lm_similaritiry(e1, e2) for e1, e2 in zip(t1, t2)):
                cooccurrence_count += 1
    
    recall = cooccurrence_count/len(expected_metarelations)
    precision = cooccurrence_count/len(metarelations)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)


## subsumption hierarchy

In [25]:
subsumption_threshold = 0.99

In [26]:
llm_pipelines = [
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            CTsToConceptExtraction(),
            SubsumptionHierarchisation(threshold=subsumption_threshold)
        ],
        corpus_loader=corpus_loader
    ),
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            SemanticBasedEnrichment(
                threshold=.9
            ),
            SynonymConceptExtraction(),
            SubsumptionHierarchisation(threshold=subsumption_threshold)
        ],
            corpus_loader=corpus_loader
    ),
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            ),
            SubsumptionHierarchisation(threshold=subsumption_threshold)
        ],
        corpus_loader=corpus_loader
    )
]

for pipeline in llm_pipelines:
    free_gpu()
    pipeline.run()

ct_pipeline, syn_pipeline, agg_pipeline = llm_pipelines


[2024-07-15 14:05:31,807] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for embedding_model parameter, default will be set to all-mpnet-base-v2.]
[2024-07-15 14:05:31,808] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for metric option, default will be set to cosine.]
100%|██████████| 1081/1081 [00:00<00:00, 36539.52it/s]


In [38]:
found_metarelations = pipeline.kr.metarelations

metarelations_embeding = [(model.encode(metarelation.source_concept.label), model.encode(metarelation.destination_concept.label)) for metarelation in found_metarelations]
expected_metarelation_embeding = [(model.encode(metarelation.source_concept.label), model.encode(metarelation.destination_concept.label)) for metarelation in expected_metarelations]


get_metarelation_ratio(metarelations_embeding, expected_metarelation_embeding)

(0.02857142857142857, 0.045454545454545456, 0.03508771929824561)

In [ ]:
display_metarelation(agg_pipeline.kr)

35 MetaRelations in KR:
bulges is_generalised_by steel plate
bulges is_generalised_by pits
block irregular distribution is_generalised_by inclusion
metal surface is_generalised_by inclusion
fish scale shape is_generalised_by inclusion
strip shape is_generalised_by inclusion
mechanical lubricant is_generalised_by oil spot
block irregular distribution is_generalised_by spots
punching defects is_generalised_by punching
punching defects is_generalised_by product
fold is_generalised_by crease
mechanical lubricant is_generalised_by product
metal surface is_generalised_by spots
fish scale shape is_generalised_by spots
plaque is_generalised_by strip surface
strip shape is_generalised_by spots
drying is_generalised_by water spot
mechanical failure is_generalised_by punching
mechanical failure is_generalised_by product
silk spot is_generalised_by strip surface
density is_generalised_by strip surface
defect type is_generalised_by strip surface
contamination is_generalised_by oil spot
contaminatio

## llm hierarchisation

In [27]:
from spacy.tokens import Doc

from olaf.data_container.metarelation_schema import Metarelation

doc_context_max_len = 4000

def find_concept_by_label( label: str, concepts: Set[Concept]) -> Concept:
    """Find a concept based on its label.

    Parameters
    ----------
    label: str
        The label of the wanted concept.
    concepts: Set[Concept]
        The set of concepts to be searched.

    Returns
    -------
    Concept
        The concept with the wanted label.
    """
    selected_concept = None
    for concept in concepts:
        if concept.label == label:
            selected_concept = concept
            break
    return selected_concept

def create_concepts_description(concepts: Set[Concept]) -> str:
        """Create concepts textual description.

        Parameters
        ----------
        concepts: Set[Concept]
            Concepts to describe.

        Returns
        -------
        str
            Textual description of the concepts.
        """
        concepts_description = "Concepts:\n"
        for concept in concepts:
            lrs = [
                lr.label
                for lr in concept.linguistic_realisations
                if not (lr.label == concept.label)
            ]
            if len(lrs):
                concepts_description += f"{concept.label} ({', '.join(lrs)})\n"
            else:
                concepts_description += f"{concept.label}\n"
        return concepts_description

def generate_doc_context( popular_docs: Set[Doc]) -> str:
        """Create context from documents with a fix size.

        Parameters
        ----------
        popular_docs: Set[Doc]
            spaCy docs to fill the context with.

        Returns
        -------
        str
            Concatenation of document contents up to a fixed size.
        """
        context = ""
        for doc in popular_docs:
            if len(doc.text) < doc_context_max_len - len(context):
                context += doc.text
                context += " "
            else:
                context += doc.text[: doc_context_max_len - len(context)]
                break
        return context

def create_hierarchisation_prompt(agg_pipeline, domain=""):
    popular_docs = set(agg_pipeline.corpus)
    context = generate_doc_context(popular_docs)
    concepts_description = create_concepts_description(pipeline.kr.concepts)
    domain = f"of {domain}" if len(domain)  else ""

    prompt = """You are an helpful assistant helping building an ontology {domain}.
        Based on the context given, define if there is a hierarchy between the listed concepts.
        The result should be given as a python list of list of string with double quotes.

        Here is an example. Concepts: animal, mammal, dog(canine), flower
        [["mammal","is_generalised_by","animal"], ["dog","is_generalised_by","mammal"], ["dog","is_generalised_by","animal"]]
    
        Context: {doc_context}
        {concepts_description}""".format(domain=domain, doc_context=context, concepts_description=concepts_description)
    return prompt

def create_metarelations(
     llm_output: str, concepts: Set[Concept]
) -> Set[Metarelation]:
    """Create metarelations based on the LLM output.

    Parameters
    ----------
    llm_output: str
        Answer of the LLM for the hierarchy.
    concepts: Set[Concept]
        The set of existing concepts.

    Returns
    -------
    Set[Metarelation]
        The metarelations created.
    """
    metarelations = set()
    try:
        for meta_tuple in llm_output:
            source_concept = find_concept_by_label(meta_tuple[0], concepts)
            destination_concept = find_concept_by_label(
                meta_tuple[2], concepts
            )
            if source_concept is not None and destination_concept is not None and meta_tuple[1] == "is_generalised_by":
                new_metarelation = Metarelation(
                    source_concept, destination_concept, "is_generalised_by"
                )
                metarelations.add(new_metarelation)
    except (SyntaxError, ValueError):
        logger.error(
            """LLM generator output is not in the expected format. 
            The metarelations can not be extracted."""
        )
    return metarelations

In [28]:
domain1 = ""
ct_prompt1 = create_hierarchisation_prompt(ct_pipeline, domain=domain1)
print(ct_prompt1)

You are an helpful assistant helping building an ontology .
        Based on the context given, define if there is a hierarchy between the listed concepts.
        The result should be given as a python list of list of string with double quotes.

        Here is an example. Concepts: animal, mammal, dog(canine), flower
        [["mammal","is_generalised_by","animal"], ["dog","is_generalised_by","mammal"], ["dog","is_generalised_by","animal"]]
    
        Context:     Oil spot: An oil spot is usually caused by the contamination of mechanical lubricant, which will affect the appearance of the product.
 Each defect type is described in detail, explaining how it appears on the steel strip surface and the reasons behind its occurrence:
     Inclusion: Inclusion is a typical defect of metal surface defects, usually showing small spots, fish scale shape, strip shape, block irregular distribution in the strip of the upper and lower surface (global or local), and is often accompanied by rough 

In [29]:
import ast



llm_output1 = [
    ["weld line", "is_generalised_by", "welding line"],
    ["crease", "is_generalised_by", "defect"],
    ["mechanical lubricant", "is_part_of", "mechanical system"],
    ["roller", "is_part_of", "production line"],
    ["pressure", "is_related_to", "roller"],
    ["product specifications", "is_part_of", "product"],
    ["damage", "is_generalised_by", "defect"],
    ["weld line", "is_part_of", "welding process"],
    ["drying", "is_part_of", "production process"],
    ["product", "is_related_to", "production line"]
]

create_metarelations(llm_output1, ct_pipeline.kr.concepts)

{meta : (weld line, is_generalised_by, welding line)}

In [30]:
domain2 = "technical documentation of quality defects"
ct_prompt2 = create_hierarchisation_prompt(ct_pipeline, domain=domain2)
print(ct_prompt2)

You are an helpful assistant helping building an ontology of technical documentation of quality defects.
        Based on the context given, define if there is a hierarchy between the listed concepts.
        The result should be given as a python list of list of string with double quotes.

        Here is an example. Concepts: animal, mammal, dog(canine), flower
        [["mammal","is_generalised_by","animal"], ["dog","is_generalised_by","mammal"], ["dog","is_generalised_by","animal"]]
    
        Context:     Oil spot: An oil spot is usually caused by the contamination of mechanical lubricant, which will affect the appearance of the product.
 Each defect type is described in detail, explaining how it appears on the steel strip surface and the reasons behind its occurrence:
     Inclusion: Inclusion is a typical defect of metal surface defects, usually showing small spots, fish scale shape, strip shape, block irregular distribution in the strip of the upper and lower surface (global 

In [31]:
llm_output2 = [
    ["fish scale shape", "is_generalised_by", "inclusion"],
    ["plaque", "is_generalised_by", "silk spot"],
    ["rolled pit", "is_generalised_by", "defect type"],
    ["pits", "is_generalised_by", "rolled pit"],
    ["bulges", "is_generalised_by", "rolled pit"],
    ["silk spot", "is_generalised_by", "defect type"],
    ["spacing", "is_generalised_by", "crease"],
    ["punching defects", "is_generalised_by", "defect type"],
    ["punching defects", "is_generalised_by", "punching"],
    ["oil spot", "is_generalised_by", "defect type"],
    ["contamination", "is_generalised_by", "oil spot"],
    ["rolled pit", "is_generalised_by", "work roll"],
    ["rolled pit", "is_generalised_by", "tension roll"],
    ["crescent gap", "is_generalised_by", "defect type"],
    ["weld line (welding line)", "is_generalised_by", "production line"],
    ["weld line (welding line)", "is_generalised_by", "cutting"],
    ["waist folding", "is_generalised_by", "defect type"],
    ["waist folding", "is_generalised_by", "fold"],
    ["waist folding", "is_generalised_by", "local deformation"],
    ["creasing", "is_generalised_by", "defect type"],
    ["creasing", "is_generalised_by", "fold"],
    ["creasing", "is_generalised_by", "local yield"],
    ["water spot", "is_generalised_by", "defect type"],
    ["water spot", "is_generalised_by", "drying"],
    ["water spot", "is_generalised_by", "oil spot"],
    ["inclusion", "is_generalised_by", "defect type"],
    ["inclusion", "is_generalised_by", "spots"],
    ["inclusion", "is_generalised_by", "block irregular distribution"],
    ["mechanical failure", "is_generalised_by", "punching defects"],
    ["mechanical lubricant", "is_generalised_by", "oil spot"],
    ["strip surface (steel strip surface)", "is_generalised_by", "metal surface"],
    ["appearance", "is_generalised_by", "product"],
    ["work roll", "is_generalised_by", "production line"],
    ["tension roll", "is_generalised_by", "production line"],
    ["strip shape", "is_generalised_by", "inclusion"],
    ["spots", "is_generalised_by", "inclusion"],
    ["pressure", "is_generalised_by", "roller"],
    ["density", "is_generalised_by", "silk spot"],
    ["temperature", "is_generalised_by", "roller"],
    ["fold", "is_generalised_by", "waist folding"],
    ["fold", "is_generalised_by", "crease"],
    ["crease", "is_generalised_by", "defect type"],
    ["creasing", "is_generalised_by", "strip surface (steel strip surface)"],
    ["uncoiling process", "is_generalised_by", "crease"],
    ["block irregular distribution", "is_generalised_by", "inclusion"],
    ["local deformation", "is_generalised_by", "waist folding"],
    ["local yield", "is_generalised_by", "crease"],
    ["mechanical failure", "is_generalised_by", "production line"],
    ["product specifications", "is_generalised_by", "product"],
    ["processes", "is_generalised_by", "production line"],
    ["product", "is_generalised_by", "production line"],
    ["plate (steel plate)", "is_generalised_by", "metal surface"],
    ["damage", "is_generalised_by", "rolled pit"],
    ["defect type", "is_generalised_by", "metal surface"],
    ["rough pockmarked surfaces", "is_generalised_by", "inclusion"],
    ["wrinkles", "is_generalised_by", "waist folding"],
    ["low-carbon", "is_generalised_by", "waist folding"]
]


metarelations = create_metarelations(llm_output2, ct_pipeline.kr.concepts)

# Add metric to hierarchy metarelation

In [33]:




metarelations_embeding = [(model.encode(metarelation.source_concept.label), model.encode(metarelation.destination_concept.label)) for metarelation in metarelations]
expected_metarelation_embeding = [(model.encode(metarelation.source_concept.label), model.encode(metarelation.destination_concept.label)) for metarelation in expected_metarelations]


get_metarelation_ratio(metarelations_embeding, expected_metarelation_embeding)

(0.22448979591836735, 0.5, 0.30985915492957744)